In [1]:
import pandas as pd
import numpy as np
import psycopg2
import math
from datetime import timedelta

%matplotlib inline

con = psycopg2.connect (user = 'postgres',
                password = 'postgres',
                port = '5433',
                host = 'localhost',
                database = 'inicudb')

cur  = con.cursor()

In [2]:
uhidPatient = "RNEH.0000008375"
#uhidPatient = "RSHI.0000012984"
caseType = 'Death'
#caseType = 'Discharge'
seperator = '_'

In [3]:
data = pd.read_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_1.csv')

/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data['series'] = data.hour_series

In [5]:
data.columns

Index(['Unnamed: 0', 'actual_DOB', 'uhid', 'dischargeddate', 'actual_DOA',
       'gender', 'birthweight', 'birthlength', 'birthheadcircumference',
       'inout_patient_status', 'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'dischargestatus', 'birthlength.1',
       'birthheadcircumference.1', 'inout_patient_status.1',
       'gestationweekbylmp.1', 'gestationdaysbylmp.1', 'baby_type.1',
       'central_temp.1', 'apgar_onemin.1', 'apgar_fivemin.1', 'apgar_tenmin.1',
       'motherage.1', 'conception_type.1', 'mode_of_delivery.1',
       'steroidname.1', 'numberofdose.1', 'los', 'day_1', 'hour_series',
       'hour', 'day_x', 'abdomen_girth', 'urine', 'stool', 'stool_passed',
       'entry_timestamp', 'day_y', 'time_divide', 'series'],
      dtype='object')

In [6]:
data.urine


0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
18424   NaN
18425   NaN
18426   NaN
18427   NaN
18428   NaN
Name: urine, Length: 18429, dtype: float64

In [7]:
data.time_divide.unique()

array([     0.   , 118841.327,   7192.408, 144699.107, 474469.605,
        64828.19 ])

In [8]:
def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

def split_hour(x):
    try:
        t=str(x)
        return t.split(":")[0]
    except:
        pass
    
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass
    

In [9]:
def urine_check(x):
    try:
        if x > 200:
            return x/10
        else:
            return x
    except:
        pass

In [10]:
def to_float(x):
    if x == '':
        return np.nan
    else:
        return float(x)

In [11]:
dg = data.copy()

In [12]:
dg['urine'] = dg['urine'].apply(to_float)

In [13]:
dg['urine'] = dg['urine'].apply(urine_check)

In [14]:
dg['urine_per_hour'] = dg['urine']/((dg['time_divide'])/(3600))

In [15]:
dt = dg.sort_values(by = ['uhid','hour_series'])

In [16]:
ds = pd.DataFrame()

In [17]:
for i in dt.uhid.unique():
    x = dt[dt['uhid']==i]
    x['urine_per_hour'].fillna(method='bfill',inplace=True)
    ds = ds.append(x,ignore_index=True)
       

In [18]:
cur1 = con.cursor()
cur1.execute("SELECT uhid,creationtime,modificationtime,ph,entrydate FROM apollo.nursing_bloodgas where uhid ='"+ uhidPatient+"'")
cols1 = list(map(lambda x: x[0],cur1.description))
ph = pd.DataFrame(cur1.fetchall(),columns=cols1)

In [19]:
ph

,uhid,creationtime,modificationtime,ph,entrydate


In [20]:
def ph_func(x):
    try:
        p = float(x)
        if p>=6.80 and p<=7.8:
            return p
        else:
            return None
    except:
        pass

In [21]:
ph

,uhid,creationtime,modificationtime,ph,entrydate


In [22]:
ph['new_ph'] = ph.ph.apply(ph_func)

In [23]:
ph.dropna(subset=['new_ph'],inplace=True)

In [24]:
ph_df = pd.DataFrame()
for i in ph.uhid.unique():
    x = ph[ph['uhid']==i]
    ph_df = ph_df.append(x,ignore_index=True)

In [25]:
ph

,uhid,creationtime,modificationtime,ph,entrydate,new_ph


In [26]:
ph_df

""


In [27]:
#ph_df = pd.DataFrame(columns = ph.columns)


In [28]:
ph_df

""


In [29]:
if len(ph_df.columns)>0:
    ph_df['hour'] = ph_df.entrydate.apply(split_hour)
    ph_df.drop_duplicates(subset=['uhid','hour'],keep='last',inplace=True)
else:
    ph_df = ph.copy()
    ph_df['uhid']=uhidPatient

In [30]:
ph_df

,uhid,creationtime,modificationtime,ph,entrydate,new_ph


In [31]:
ds

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y,time_divide,series,urine_per_hour
0,0,2018-09-12 08:38:00.000,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00.000,Female,845.0,NaN,NaN,In Born,...,2018-09-12,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-09-12 09:00:00.000,0.757312
1,1,2018-09-12 08:38:00.000,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00.000,Female,845.0,NaN,NaN,In Born,...,2018-09-12,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-09-12 09:01:00.000,0.757312
2,2,2018-09-12 08:38:00.000,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00.000,Female,845.0,NaN,NaN,In Born,...,2018-09-12,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-09-12 09:02:00.000,0.757312
3,3,2018-09-12 08:38:00.000,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00.000,Female,845.0,NaN,NaN,In Born,...,2018-09-12,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-09-12 09:03:00.000,0.757312
4,4,2018-09-12 08:38:00.000,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00.000,Female,845.0,NaN,NaN,In Born,...,2018-09-12,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-09-12 09:04:00.000,0.757312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,18424,2018-09-12 08:38:00.000,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00.000,Female,845.0,NaN,NaN,In Born,...,2018-09-25,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-09-25 04:04:00.000,NaN
18425,18425,2018-09-12 08:38:00.000,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00.000,Female,845.0,NaN,NaN,In Born,...,2018-09-25,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-09-25 04:05:00.000,NaN
18426,18426,2018-09-12 08:38:00.000,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00.000,Female,845.0,NaN,NaN,In Born,...,2018-09-25,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-09-25 04:06:00.000,NaN
18427,18427,2018-09-12 08:38:00.000,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00.000,Female,845.0,NaN,NaN,In Born,...,2018-09-25,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-09-25 04:07:00.000,NaN


In [32]:
ph_df

,uhid,creationtime,modificationtime,ph,entrydate,new_ph


In [33]:
type(ds.uhid[0])

str

In [34]:
ds['uhid'] = ds['uhid'].astype(str)


In [35]:
ss = pd.merge(ds,ph_df,on=['uhid','hour'],how='left')
ss = ss.drop_duplicates(subset=['uhid','hour_series'],keep='first')

KeyError: 'hour'

In [ ]:
ss

In [ ]:
ss

In [ ]:
len(ss.uhid.unique())

In [ ]:
cur3 = con.cursor()
cur3.execute("SELECT t1.uhid,t1.entrydate,t1.rbs,t1.skintemp,t1.centraltemp  FROM apollo.nursing_vitalparameters AS t1 where t1.uhid = '"+uhidPatient+"'")
cols3 = list(map(lambda x: x[0], cur3.description))
nv = pd.DataFrame(cur3.fetchall(),columns=cols3)

In [ ]:
nv.drop_duplicates(subset=['uhid','entrydate'],keep='first',inplace=True)

In [ ]:
nv['hour'] = nv.entrydate.apply(split_hour)

In [ ]:
nv

In [ ]:
ds

In [ ]:
nv.hour.iloc[0]

In [ ]:
ss.uhid = ss.uhid.astype(str)

In [ ]:
ss.hour.iloc[0]

In [ ]:
s1 = pd.merge(ss,nv,on = ['uhid','hour'], how='left')

In [ ]:
len(s1.uhid.unique())

In [ ]:
s1.drop_duplicates(subset=['uhid','hour_series'],keep='first',inplace=True)

In [ ]:
s1

In [ ]:
def temp_a(x,y):
    if x>0:
        return x
    else:
        return y

In [ ]:
s1['temp'] = s1.apply(lambda x: temp_a(x['centraltemp'], x['skintemp']), axis=1)

In [ ]:
cur2 = con.cursor()
cur2.execute("SELECT t1.uhid,t1.visitdate,t1.visittime,t1.currentdateweight, t1.currentdateheight  FROM apollo.baby_visit AS t1 where t1.uhid = '"+uhidPatient+"'")
cols2 = list(map(lambda x: x[0], cur2.description))
bv = pd.DataFrame(cur2.fetchall(),columns=cols2)

In [ ]:
bv

In [ ]:
bv.sort_values('visitdate',inplace=True)

In [ ]:
def stamp_1(x,y):
    return str(x) +" "+ str(y).split(":")[0]

In [ ]:
bv

In [ ]:
bv['hour'] = bv.apply(lambda x: stamp_1(x['visitdate'], x['visittime']), axis=1)

In [ ]:
def to_str(x):
    return (str(x))

In [ ]:
bv['day'] = bv['visitdate'].apply(to_str)

In [ ]:
bv.drop_duplicates(subset=['uhid','visitdate'],keep='last',inplace=True)

In [ ]:
s1['day'] = s1['hour_series'].apply(split_date_1)

In [ ]:
s1.day.unique()

In [ ]:
s1

In [ ]:
s2 = pd.merge(s1,bv,on = ['uhid','day'], how='left')

In [ ]:
len(s2.uhid.unique())

In [ ]:
#Meds



In [ ]:
cur7= con.cursor()
cur7.execute("SELECT t1.uhid,t1.startdate,t1.medicineorderdate,t1.medicinename,t1.medicationtype,t2.typevalue FROM apollo.baby_prescription AS t1 LEFT JOIN apollo.ref_medtype AS t2 ON t1.medicationtype = t2.typeid WHERE (t1.medicationtype = 'TYPE0001' OR t1.medicationtype = 'TYPE0009' OR t1.medicationtype = 'TYPE0004') and (t1.uhid ='"+uhidPatient+"')")
cols7 = list(map(lambda x: x[0], cur7.description))
meds = pd.DataFrame(cur7.fetchall(),columns=cols7)

In [ ]:
meds

In [ ]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])


In [ ]:
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)


In [ ]:
meds_dummies

In [ ]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)
s2.drop(['hour_x','hour_y'],axis=1,inplace=True)

In [ ]:
s2['hour'] = s2.hour_series.apply(split_hour)

In [ ]:
meds_dummies

In [ ]:
s2

In [ ]:
s3 = pd.merge(s2,meds_dummies,on = ['uhid','hour'], how='left')

In [ ]:
s3['day'] = s3['hour_series'].apply(split_date_1)

In [ ]:
s3

In [ ]:
#PN
cur8= con.cursor()
cur8.execute("SELECT t1.uhid,t1.entrydatetime,t1.totalparenteralvolume,t1.total_intake FROM apollo.babyfeed_detail AS t1 where t1.uhid = '"+uhidPatient+"'")
cols8 = list(map(lambda x: x[0], cur8.description))
pn = pd.DataFrame(cur8.fetchall(),columns=cols8)

In [ ]:
pn['hour'] = pn.entrydatetime.apply(split_hour)
pn['total_intake'] = pn['total_intake'].astype(float)
pn['tpn-tfl'] = pn['totalparenteralvolume']/pn['total_intake']

In [ ]:
s4 = pd.merge(s3,pn,on = ['uhid','hour'], how='left')

In [ ]:
s4

In [ ]:
len(s4.uhid.unique())

In [ ]:
s4.drop_duplicates(subset=['uhid','hour_series'],inplace=True)

In [ ]:
q = pd.DataFrame()

In [ ]:
for i in s4.uhid.unique():
    x = s4[s4['uhid']==i]
    x['currentdateheight'].fillna(method='ffill',inplace=True)
    x['currentdateweight'].fillna(method='ffill',inplace=True)
    q = q.append(x,ignore_index=True)

In [ ]:
def abd(x):
    
    try:   
        
        if x>200:
            return x/10
        else:
            return x
    except:
        pass
    

In [ ]:
q['abdomen_girth'] = pd.to_numeric(q['abdomen_girth'], errors='coerce')
q['abdomen_girth'] = q['abdomen_girth'].apply(abd)

In [ ]:
dt = pd.DataFrame()
for i in q.uhid.unique():
    x = q[q['uhid']==i]
    #for j in y['day'].unique():
    #x = y[y['day']==j]
    x['abdomen_girth'].fillna(method='bfill',inplace=True)
    x['currentdateweight'].fillna(method='bfill',inplace=True)
    x['currentdateheight'].fillna(method='bfill',inplace=True)
    x['totalparenteralvolume'].fillna(method='ffill',inplace=True)
    x['total_intake'].fillna(method='ffill',inplace=True)
    x['tpn-tfl'].fillna(method='ffill',inplace=True)
    dt = dt.append(x,ignore_index=True)

In [ ]:
dt.urine_per_hour.max()

In [ ]:
dt.urine_per_hour.unique()

In [ ]:
a = dt.replace([np.inf, -np.inf], np.nan)

In [ ]:
def abd_2(x):
    if x < 0:
        return ((-1)*x)/10
    elif x > 200:
        return x/10
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_2)

def abd_3(x):
    if x < 15:
        return np.nan
    elif x > 50:
        return np.nan
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_3)

def upm(x):
    if x > 100:
        return np.nan
    else:
        return x
    
a['urine_per_hour'] = a['urine_per_hour'].apply(upm)

def temp_f_c(x):
    if x > 90:
        return (x - 32)*(5/9)
    else:
        return x
    
a['temp'] = a['temp'].apply(temp_f_c)

def weight_correct(x):
    if x<10:
        return x*100
    elif x>5000:
        return x/10
    else:
        return x
    
def weight_correct_2(x):
    if x<300:
        return x*10
    elif x>5000:
        return x/10
    else:
        return x
    
a.currentdate = a.currentdateweight.apply(weight_correct)
a.currentdate = a.currentdateweight.apply(weight_correct_2)

def height_correct(x):
    if x < 10:
        return np.nan
    else:
        return x
    
def rbs_correct(x):
    if x > 700:
        return np.nan
    else:
        return x
    
a['currentdateheight'] = a['currentdateheight'].apply(height_correct)

a['rbs'] = a['rbs'].apply(rbs_correct)



In [ ]:
len(a.uhid.unique())

In [ ]:
data = a.copy()

In [ ]:
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass

In [ ]:
data['day'] = data['hour_series'].apply(split_date_1)

In [ ]:
def to_date(x):
    return pd.to_datetime(x)

In [ ]:
data['hour_series'] = data['hour_series'].apply(to_date)

In [ ]:
len(data.uhid.unique())

In [ ]:
data.to_csv(caseType+seperator+uhidPatient+seperator+"intermittent_checkpoint_2.csv")

In [ ]:
df = pd.DataFrame()
for i in data.uhid.unique():
    x = data[data['uhid']==i]
    x.rbs.fillna(method='ffill',inplace=True)
    x.currentdateweight.fillna(method='ffill',inplace=True)
    x.currentdateheight.fillna(method='ffill',inplace=True)
    n = math.ceil(len(x)/24)+1
    start_date = pd.to_datetime(x['day'].iloc[0]+" " + "08:00:00") - timedelta(hours=24)
    for i in range(int(n)):
        
       
        y = x[(x['hour_series']>=start_date + timedelta(hours=24*i)) & (x['hour_series']<=start_date + timedelta(hours=24*(i+1)))]
        y['stool_day_total'] = y['stool_passed'].sum()
        df =df.append(y,ignore_index=True)


In [ ]:
df.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_3.csv')

In [ ]:
test = df.copy()

In [ ]:
dq = pd.DataFrame()

In [ ]:
for i in test.uhid.unique():
    x = test[test['uhid']==i]
    x.abdomen_girth.fillna(method='ffill',inplace=True)
    dq = dq.append(x, ignore_index=True)

In [ ]:
final = pd.DataFrame()

In [ ]:
for i in dq.uhid.unique():
    x = dq[dq['uhid']==i]
    x['abd_difference_y'] = x['abdomen_girth'] - x['abdomen_girth'].shift(periods=1)
    final = final.append(x,ignore_index=True)

In [ ]:
final.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_4.csv')

In [ ]:
cur1 = con.cursor()
cur1.execute("SELECT t1.uhid,t1.starttime,t1.pulserate, t1.ecg_resprate, t1.spo2, t1.heartrate, t2.dischargestatus,t1.mean_bp,t1.sys_bp,t1.dia_bp  FROM apollo.device_monitor_detail_dump AS t1 LEFT JOIN apollo.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE (t2.dischargestatus = 'Discharge' AND t2.dateofadmission > '2018-07-01' and t2.uhid = '"+ uhidPatient+"') OR (t2.dischargestatus = '"+caseType+"' AND t2.dateofadmission > '2018-07-01' and t2.uhid = '"+ uhidPatient+"');")
cols1 = list(map(lambda x: x[0], cur1.description))
ds = pd.DataFrame(cur1.fetchall(),columns=cols1)
ds.sort_values('starttime',inplace=True)

In [ ]:
ds

In [ ]:
test = ds.drop_duplicates(subset=['uhid','starttime'],keep='first')

In [ ]:
cur_vent = con.cursor()
cur_vent.execute("SELECT t1.uhid,t1.start_time,t1.creationtime,t1.peep, t1.pip,t1.map ,t1.tidalvol, t1.minvol,t1.ti,t1.fio2 FROM apollo.device_ventilator_detail_dump AS t1 LEFT JOIN apollo.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE (t2.dischargestatus = 'Discharge' OR t2.dischargestatus = '"+caseType+"') and t2.uhid = '"+uhidPatient+"';")
cols_vent = list(map(lambda x: x[0], cur_vent.description))
ventilator_cont = pd.DataFrame(cur_vent.fetchall(),columns=cols_vent)

test_vent = ventilator_cont.drop_duplicates(subset=['uhid','start_time'],keep='first')

def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

test['date'] = test.starttime.apply(split_date)
test_vent['date'] = test_vent.start_time.apply(split_date)

In [ ]:
cont_data = pd.merge(test,test_vent,on=['uhid','date'],how='left',copy=False)
test_cont = cont_data.drop_duplicates(subset=['uhid','starttime','heartrate'],keep='first')


In [ ]:
test_cont['hour_series'] = test_cont['date'].apply(split_hour)

In [ ]:
def to_str(x):
    return str(x)

In [ ]:
test_cont['ref_hour'] = test_cont['hour_series'].apply(to_str)
final['ref_hour'] = final['hour_series'].apply(to_str)

In [ ]:
def con_time(x):
    return str(x).split("+")[0].split(":")[0] + ":" +str(x).split("+")[0].split(":")[1]

In [ ]:
test_cont['cont_time'] = test_cont.starttime.apply(con_time)

In [ ]:
def con_time_2(x):
    return str(x).split(":")[0] +":"+str(x).split(":")[1]

In [ ]:
final['cont_time'] = final.ref_hour.apply(con_time_2)

In [ ]:
qw = pd.merge(final,test_cont, on=['uhid','cont_time'],how='left')

In [ ]:
#final[final['cont_time']=='2019-09-30 11:18']

In [ ]:
#qw.dropna(subset=['series'],inplace=True)

In [ ]:
qw.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_5.csv')